In [1]:
import pandas as pd
import numpy as np

# The product names are not standard, so we'll group by brand name
reviewsDF = pd.read_csv('Amazon_Unlocked_Mobile.csv')
# normalize brand names
reviewsDF['Brand Name'] = reviewsDF['Brand Name'].str.lower()
reviews = reviewsDF.Reviews.tolist()
brands = reviewsDF['Brand Name'].tolist()
ratings = reviewsDF['Rating'].tolist()
print("Loaded " + str(len(reviews)) + " reviews.")

Loaded 413840 reviews.


In [2]:
reviewsDF.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",samsung,199.99,5,Very pleased,0.0
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0


In [3]:
import nltk
def get_all_phrases_containing_tar_wrd(target_word, tar_passage, left_margin = 5, right_margin = 5):
    
    ## Create list of tokens using nltk function
    tokens = nltk.word_tokenize(tar_passage)
    tokens = [x for x in tokens if len(x)>2] 
    ## Create the text of tokens
    text = nltk.Text(tokens)
 
    ## Collect all the index or offset position of the target word
    c = nltk.ConcordanceIndex(text.tokens, key = lambda s: s.lower())
 
    ## Collect the range of the words that is within the target word by using text.tokens[start;end].
    ## The map function is use so that when the offset position - the target range < 0, it will be default to zero
    
    concordance_txt = ([text.tokens[  list(map(lambda x: x-5 if (x-left_margin)>0 else 0,[offset]))[0]:offset+right_margin] for offset in c.offsets(target_word)])

    ## join the sentences for each of the target phrase and return it
    return [''.join([x+' ' for x in con_sub]) for con_sub in concordance_txt]

In [4]:
from nltk import sent_tokenize

## Select the aspects we believe are relevant 
relevant_aspects = ['battery', 'screen', 'camera', 'performance']

## Make lists for each of the columns that we will have in our new dataframe
phrases = []
brand = []
rating = []
aspects = []
review_num = []
counter = 0
# Loop through all the reviews
for review in reviews:
    try:
        # Break the reviews in sentences and loop through them
        for sentence in sent_tokenize(review):
            for  important_word in relevant_aspects:
                # Get all the phrases containing one of the topic names
                # Create append list of phrases, aspects, ratings etc to create a DF
                phrases_in_sentence = get_all_phrases_containing_tar_wrd(important_word, sentence, left_margin = 5, right_margin = 5) 
                for phrase in phrases_in_sentence:
                    brand.append(brands[counter])
                    rating.append(ratings[counter])
                    aspect = 'other'
                    for imp_word in relevant_aspects:
                        if imp_word in phrase.lower():
                            aspect = imp_word
                    aspects.append(aspect)
                    phrases.append(phrase)
                    review_num.append(counter)
    except:
        pass
    counter = counter + 1

print('Extracted ' + str(len(phrases)) + ' phrases from the reviews')

# Create the phrases dataframe
phrasesDF = pd.DataFrame()
phrasesDF['brand'] = brand 
phrasesDF['phrase'] = phrases
phrasesDF['rating'] = rating
phrasesDF['aspect'] = aspects
phrasesDF['review_id'] = review_num
phrasesDF.head()

Extracted 0 phrases from the reviews


,brand,phrase,rating,aspect,review_id
